In [68]:
import matplotlib
matplotlib.use('Agg')
import json
from datetime import datetime
import sys
import os
import mlflow
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from etl import EDAdataset
sys.path.append(os.path.abspath('..'))  # Ajusta la ruta según la ubicación de 'src'
from utils.conexion import SQLConnection
from feature_engineer import PreprocesadorTexto
from prefect import flow, get_run_logger, task
from prefect.artifacts import create_markdown_artifact, create_table_artifact
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from train_with_mlflow_optuna import TrainMlflowOptuna
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

def build_pipeline(model):
    # Transformador para extraer la columna de texto
    text_transformer = Pipeline([
        ('tfidf', TfidfVectorizer(decode_error='ignore'))
    ])

    # Transformador para columnas numéricas
    numeric_transformer = 'passthrough'

    # ColumnTransformer para aplicar transformaciones específicas
    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_transformer, 'concatenada'),
            ('num', numeric_transformer, ['sexo_codificado'])
        ]
    )

    # Pipeline completo
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    return pipeline

In [70]:
from scipy.sparse import hstack, csr_matrix

@task(name="etl_dataset", retries=2, retry_delay_seconds=10)
def task_generate_data(nregistros: int = 10000, db_server=os.getenv("DB_SERVER"), 
                       db_name=os.getenv("DB_NAME"),
                       db_driver=os.getenv("DB_DRIVER"),
                       params = {"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}, 
                       sql_path = os.path.join("..", "..", "sql_queries", "queries.sql")) -> pd.DataFrame:
    """
    Generate synthetic user data for training.
    
    Args:
        nregistros: Number of registros to generate
        
    Returns:
        Generated dataframe
    """
    logger = get_run_logger()
    logger.info(f"Generating {nregistros} registros...")
    
    # Generate data
    load_dotenv()
    db_server=os.getenv("DB_SERVER")
    db_name=os.getenv("DB_NAME")
    db_driver=os.getenv("DB_DRIVER")
    sqlconection = SQLConnection(sql_path=sql_path, db_server=db_server, db_name=db_name, db_driver=db_driver, params=params)
    df_conexion = sqlconection.generate_dataframe(nregistros)
    df_eda = EDAdataset(df_conexion)
    df = df_eda.dataset_eda(df_conexion)
        
    # Create summary artifact
    summary_df = pd.DataFrame({
        'Metric': ['Total Samples', 'Total Features', 'Missing Values'],
        'Value': [
            len(df),
            len(df.columns),
            df.isnull().sum().sum()
        ]
    })
    
    create_table_artifact(
        key="etl-dataframe-summary",
        table=summary_df.to_dict(orient='records'),
        description=f"ETL Data Generation Summary - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    )
    
    logger.info(f"Generated {len(df)} samples with {len(df.columns)} columns")
    return df


@task(name="Feature_Engineering", retries=2, retry_delay_seconds=10)
def task_feature_engineering(df, stopwords={
            "medico", "paciente", "psicologo", "psicologa",
            "psicologia", "psicoterapeuta", "psicoterapia", "refiere"
        }, columna_texto="concatenada", columna_sexo="sexo", columna_grupo="grupo") -> pd.DataFrame:
    """
    Apply feature engineering to the dataset.
    
    Args:
        df: Input dataframe
        
    Returns:
        Feature-engineered dataframe
    """
    logger = get_run_logger()
    logger.info("Starting feature engineering...")
    
    initial_columns = len(df.columns)
    preprocesador = PreprocesadorTexto(df, stopwords=stopwords)
    df_engineered, _ = preprocesador.procesar(columna_texto=columna_texto, columna_sexo=columna_sexo, columna_grupo=columna_grupo)
    
    # Create feature engineering summary
    feature_summary = pd.DataFrame({
        'Metric': ['Initial Features', 'Final Features', 'Features Added', 'Dataset Size'],
        'Value': [
            initial_columns,
            len(df_engineered.columns),
            len(df_engineered.columns) - initial_columns,
            f"{len(df_engineered)} rows",
        ]
    })
    
    create_table_artifact(
        key="feature-engineering-summary",
        table=feature_summary.to_dict(orient='records'),
        description="Feature Engineering Summary"
    )
    
    logger.info(f"Feature engineering complete: {initial_columns} -> {len(df_engineered.columns)} features")
    return df_engineered

@task(name="Train_Model_Optuna", retries=1, retry_delay_seconds=30)
def task_train_with_optuna(
    df: pd.DataFrame,
    model_type: str = "LogisticRegression",
    n_trials: int = 20,
    optimization_metric: str = "accuracy"
) -> tuple:
    """
    Train model with Optuna hyperparameter optimization and MLflow tracking.
    
    Args:
        df: Feature-engineered dataframe
        model_type: Type of model to train ('LogisticRegression' or 'RandomForest')
        n_trials: Number of Optuna trials
        optimization_metric: Metric to optimize
        
    Returns:
        Tuple of (best_pipeline, best_run_id, study, metrics_dict)
    """
    def build_pipeline(model):
        # Transformador para extraer la columna de texto
        text_transformer = Pipeline([
            ('tfidf', TfidfVectorizer(decode_error='ignore'))
        ])

        # Transformador para columnas numéricas
        numeric_transformer = 'passthrough'

        # ColumnTransformer para aplicar transformaciones específicas
        preprocessor = ColumnTransformer(
            transformers=[
                ('text', text_transformer, 'concatenada'),
                ('num', numeric_transformer, ['sexo_codificado'])
            ]
        )

        # Pipeline completo
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])

        return pipeline
    logger = get_run_logger()
    logger.info(f"Starting Optuna optimization for {model_type} with {n_trials} trials...")
    
    # Define Training Columns
    training_columns = ["concatenada", "sexo_codificado"]
    # Define target column
    target_column = 'grupo_codificado'

    # --- AJUSTE: Vectorizar columna de texto y concatenar con columnas numéricas ---
    # vectorizer = TfidfVectorizer(decode_error='ignore')
    # X = df[training_columns]
    # X_text = vectorizer.fit_transform(X["concatenada"].astype(str))
    # numeric_cols = [col for col in training_columns if col != "concatenada"]
    # X_numeric = X[numeric_cols].values
    # X_final = hstack([X_text, csr_matrix(X_numeric)])
    # y = df[target_column]
    # --- FIN AJUSTE ---

    # Define parameter distributions based on model type
    if model_type == "LogisticRegression":
        model_class = LogisticRegression
        param_distributions = {
            'C': ('float', 0.001, 100, True),
            'penalty': ('categorical', ['l1', 'l2']),
            'max_iter': ('int', 200, 2000),
            'solver': ('categorical', ['liblinear', 'saga'])
        }
        fixed_params = {'random_state': 42}
    elif model_type == "RandomForest":
        model_class = RandomForestClassifier
        param_distributions = {
            'n_estimators': ('int', 50, 200),
            'max_depth': ('int', 5, 30),
            'min_samples_split': ('int', 2, 15),
            'min_samples_leaf': ('int', 1, 10),
            'max_features': ('categorical', ['sqrt', 'log2'])
        }
        fixed_params = {'random_state': 42, 'n_jobs': -1}
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    # Crear pipeline base
    base_model = model_class()
    pipeline = build_pipeline(base_model)
    
    # Set up MLflow
    # mlflow.set_experiment(f"prefect_{model_type.lower()}_training")
    model_name = type(pipeline.named_steps['classifier']).__name__
    mlflow.set_experiment(f"prefect_{model_name.lower()}_training")

    mlflow.sklearn.autolog()
    
    # Create trainer
    trainer = TrainMlflowOptuna(
        df=df,
        target_column=target_column,
        model_class=pipeline,
        test_size=0.3,
        n_trials=n_trials,
        optimization_metric=optimization_metric,
        param_distributions=param_distributions,
        model_params=fixed_params
    )
    
    # Run optimization
    best_pipeline, best_run_id, study = trainer.train()

    # --- AJUSTE: Validación usando datos vectorizados ---
    # X_test = df[training_columns][:100]
    # X_test_text = vectorizer.transform(X_test["concatenada"].astype(str))
    # X_test_numeric = X_test[numeric_cols].values
    # X_test_final = hstack([X_test_text, csr_matrix(X_test_numeric)])
    # y_test = df[target_column][:100]
    # y_pred = best_pipeline.predict(X_test_final)
    X_test = df[training_columns][:100]
    y_test = df[target_column][:100]
    y_pred = best_pipeline.predict(X_test)


    metrics_dict = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1': f1_score(y_test, y_pred, average='weighted')
    }

    logger.info(f"Optimization complete! Best {optimization_metric}: {study.best_value:.4f}")
    logger.info(f"Best parameters: {study.best_params}")
    logger.info(f"MLflow Run ID: {best_run_id}")

    return best_pipeline, best_run_id, study, metrics_dict

@task(name="Create_Model_Report", retries=1)
def task_create_model_report(
    model_type: str,
    best_run_id: str,
    study,
    metrics_dict: dict,
    n_trials: int
) -> None:
    """
    Create comprehensive model training report as markdown artifact.
    
    Args:
        model_type: Type of model trained
        best_run_id: MLflow run ID
        study: Optuna study object
        metrics_dict: Dictionary of validation metrics
        n_trials: Number of trials performed
    """
    logger = get_run_logger()
    logger.info("Creating model training report...")
    
    # Create markdown report
    markdown_content = f"""
# Model Training Report - {model_type}

## Training Summary
- **Date**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Model Type**: {model_type}
- **Number of Trials**: {n_trials}
- **MLflow Run ID**: `{best_run_id}`

## Optimization Results
- **Best Score**: {study.best_value:.4f}
- **Optimization Metric**: {study.trials[0].user_attrs.get('metric_name', 'accuracy') if study.trials else 'N/A'}
- **Total Trials Completed**: {len([t for t in study.trials if t.state.name == 'COMPLETE'])}

## Best Hyperparameters
```python
{json.dumps(study.best_params, indent=2)}
```

## Validation Metrics
| Metric | Score |
|--------|-------|
| Accuracy | {metrics_dict['accuracy']:.4f} |
| Precision | {metrics_dict['precision']:.4f} |
| Recall | {metrics_dict['recall']:.4f} |
| F1 Score | {metrics_dict['f1']:.4f} |

## Top 5 Trials
| Trial | Score | Parameters |
|-------|-------|------------|
"""
    
    # Add top 5 trials
    sorted_trials = sorted(study.trials, key=lambda t: t.value if t.value else 0, reverse=True)[:5]
    for trial in sorted_trials:
        if trial.value:
            params_str = ', '.join([f"{k}={v}" for k, v in list(trial.params.items())[:3]])
            markdown_content += f"| {trial.number} | {trial.value:.4f} | {params_str}... |\n"
    
    markdown_content += f"""

## How to Use the Model
```python
import mlflow

# Load the best model
model = mlflow.sklearn.load_model(f"runs:/{best_run_id}/model")

# Make predictions
predictions = model.predict(X_new)
```

## Next Steps
1. Review the model performance in MLflow UI
2. Deploy the model if metrics meet requirements
3. Monitor model performance in production
"""
    
    create_markdown_artifact(
        key="model-training-report",
        markdown=markdown_content,
        description=f"Complete Training Report for {model_type}"
    )
    
    logger.info("Model training report created successfully")


@flow(name="Train_Model_With_Optuna", log_prints=True)
def train_model_flow(
    nregistros: int = 10000,
    model_type: str = "LogisticRegression",
    n_trials: int = 20,
    optimization_metric: str = "accuracy"
):
    """
    Main Prefect flow for training models with Optuna optimization.
    
    Args:
        n_samples: Number of samples to generate
        model_type: Type of model to train
        n_trials: Number of Optuna trials
        optimization_metric: Metric to optimize
        
    Returns:
        Trained pipeline and MLflow run ID
    """
    logger = get_run_logger()
    logger.info(f"Starting training flow for {model_type}")
    
    # Task 1: Generate data
    df = task_generate_data(nregistros=nregistros)
    
    # Task 2: Feature engineering
    df_engineered = task_feature_engineering(df)
    df_engineered['concatenada'] = df_engineered['concatenada'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    
    # Task 3: Train with Optuna
    best_pipeline, best_run_id, study, metrics_dict = task_train_with_optuna(
        df_engineered,
        model_type,
        n_trials,
        optimization_metric
    )
    
    # Task 4: Create report
    task_create_model_report(
        model_type,
        best_run_id,
        study,
        metrics_dict,
        n_trials
    )
    
    logger.info(f"Training flow complete! Best model saved with run ID: {best_run_id}")
    
    # Create final summary artifact
    final_summary = pd.DataFrame({
        'Metric': ['Model Type', 'Best Score', 'MLflow Run ID', 'Total Time'],
        'Value': [
            model_type,
            f"{study.best_value:.4f}",
            best_run_id,
            f"{sum((t.datetime_complete - t.datetime_start).total_seconds() for t in study.trials if t.datetime_complete):.2f}s"
        ]
    })
    
    create_table_artifact(
        key="training-flow-summary",
        table=final_summary.to_dict(orient='records'),
        description="Final Training Flow Summary"
    )
    
    return best_pipeline, best_run_id


In [71]:
# Example 1: Train a single model
pipeline, run_id = train_model_flow(
    nregistros=5000,
    model_type="RandomForest",
    n_trials=10,
    optimization_metric="accuracy"
    )

# Example 2: Compare multiple models
# results = compare_models_flow(n_samples=5000, n_trials=10)

20:25:43.713 | INFO    | Flow run 'vague-pelican' - Beginning flow run 'vague-pelican' for flow 'Train_Model_With_Optuna'

20:25:43.717 | INFO    | Flow run 'vague-pelican' - View at http://127.0.0.1:4200/runs/flow-run/a0c6d0d1-395a-4225-8a16-6a06eed18e32

20:25:43.719 | INFO    | Flow run 'vague-pelican' - Starting training flow for RandomForest

20:25:44.034 | INFO    | Task run 'etl_dataset-c45' - Generating 5000 registros...

20:26:00.442 | INFO    | Task run 'etl_dataset-c45' - Generated 4931 samples with 7 columns

...

20:31:09.397 | ERROR   | Task run 'Train_Model_Optuna-74c' - Task run failed with exception: AttributeError("'Pipeline' object has no attribute '__name__'") - Retries are exhausted
Traceback (most recent call last):
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 871, in run_context
    yield self
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 1512, in run_task_sync
    engine.call_task_fn(txn)
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 888, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\utilities\callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ISAIAS_QUINTERO\AppData\Local\Temp\ipykernel_26904\1542256139.py", line 200, in task_train_with_optuna
    best_pipeline, best_run_id, study = trainer.train()
                                        ^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\src\training\train_with_mlflow_optuna.py", line 300, in train
    return self.train_with_optuna()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\src\training\train_with_mlflow_optuna.py", line 195, in train_with_optuna
    study_name=f"optuna_{self.model_class.__name__}"
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Pipeline' object has no attribute '__name__'

20:31:09.436 | ERROR   | Task run 'Train_Model_Optuna-74c' - Finished in state Failed("Task run encountered an exception AttributeError: 'Pipeline' object has no attribute '__name__'")

20:31:09.457 | ERROR   | Flow run 'vague-pelican' - Encountered exception during execution: AttributeError("'Pipeline' object has no attribute '__name__'")
Traceback (most recent call last):
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\flow_engine.py", line 781, in run_context
    yield self
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\flow_engine.py", line 1395, in run_flow_sync
    engine.call_flow_fn()
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\flow_engine.py", line 801, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\utilities\callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ISAIAS_QUINTERO\AppData\Local\Temp\ipykernel_26904\1542256139.py", line 346, in train_model_flow
    best_pipeline, best_run_id, study, metrics_dict = task_train_with_optuna(
                                                      ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\tasks.py", line 1139, in __call__
    return run_task(
           ^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 1739, in run_task
    return run_task_sync(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 1514, in run_task_sync
    return engine.state if return_type == "state" else engine.result()
                                                       ^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 493, in result
    raise self._raised
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 871, in run_context
    yield self
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 1512, in run_task_sync
    engine.call_task_fn(txn)
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\task_engine.py", line 888, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\.venv\Lib\site-packages\prefect\utilities\callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ISAIAS_QUINTERO\AppData\Local\Temp\ipykernel_26904\1542256139.py", line 200, in task_train_with_optuna
    best_pipeline, best_run_id, study = trainer.train()
                                        ^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\src\training\train_with_mlflow_optuna.py", line 300, in train
    return self.train_with_optuna()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\src\training\train_with_mlflow_optuna.py", line 195, in train_with_optuna
    study_name=f"optuna_{self.model_class.__name__}"
                         ^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Pipeline' object has no attribute '__name__'

20:31:09.565 | INFO    | Flow run 'vague-pelican' - Finished in state Failed("Flow run encountered an exception: AttributeError: 'Pipeline' object has no attribute '__name__'")

AttributeError: 'Pipeline' object has no attribute '__name__'